# Report: Solving Task Clarity for LLM-based Agents

## Problem Statement

The challenge of achieving task clarity for LLM-based Agents involves enabling these language model-based agents to understand and execute user-provided tasks accurately.
This report explores a current solution and outlines future work in addressing this problem.

## Example:

- **Client Input:** "Find for me a product in a website."
- **Expected Output:**
  ```python
  {'task_type': 'Information retrieval', 'scores': > 0.2, 'inference_time': < 10 s}


# Current solution: Zero-Shot Intent Classification via HuggingFace models

*   model_id: "facebook/bart-large-mnli."

In [1]:
#install library
! pip install accelerate peft bitsandbytes guidance trl py7zr
! pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yq0hb_gp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yq0hb_gp
  Resolved https://github.com/huggingface/transformers to commit a5e6df82c00cb53ffe863008cbbedd813bcc508b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
#logging Hugging Face
hf_key = "YourHuggingFaceToken"
!huggingface-cli login --token $hf_key

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
#load the model
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


In [4]:
#experiments
import asyncio
import re
import time

input = "Write a pong game in python."
candidate_labels = ["Greeting", "Information retrieval", "Sentiment analysis", "Text generation", "Code generation", "Q&A", "Summarization", "Translation"]
time_execution = time.time()
output = classifier(input, candidate_labels, multi_label=False)
time_execution = round(time.time() - time_execution, 2)
print(f"task_type: {output['labels'][0]}, scores: {round(output['scores'][0],2)}, inference_time: {time_execution}")

task_type: Code generation, scores: 0.23, inference_time: 6.29


In [5]:
async def task_classifier(client_input: str, task_types: str):
    """Classify tasks for LLM-based gent"""
    candidate_labels = [label.strip() for label in task_types.split(",")]
    time_execution = time.time()
    output = classifier(str(client_input), candidate_labels, multi_label=False)
    # output = classifier(input, candidate_labels, multi_label=False)
    time_execution = round(time.time() - time_execution, 2)
    return {"task_type": output['labels'][0], "scores": round(output['scores'][0],2), "inference_time": time_execution}

# Test the solution & results

In [6]:
# List of test cases
test_cases = [
    "Summarize the latest scientific research.",
    "Translate this English text into French.",
    "What's the weather forecast for tomorrow?",
    "Generate Python code for a simple calculator.",
    "Please provide me with the most popular video currently trending on YouTube"
]

# Run the task_classifier function for each test case
for test_case in test_cases:
    result = await task_classifier(test_case, 'Greeting, Information retrieval, Sentiment analysis, Text generation, Code generation, Q&A, Summarization, Translation')
    print(f"Input: '{test_case}'")
    print(f"Output: {result}")
    print()

Input: 'Summarize the latest scientific research.'
Output: {'task_type': 'Summarization', 'scores': 0.83, 'inference_time': 8.31}

Input: 'Translate this English text into French.'
Output: {'task_type': 'Translation', 'scores': 0.79, 'inference_time': 9.98}

Input: 'What's the weather forecast for tomorrow?'
Output: {'task_type': 'Q&A', 'scores': 0.68, 'inference_time': 6.51}

Input: 'Generate Python code for a simple calculator.'
Output: {'task_type': 'Code generation', 'scores': 0.97, 'inference_time': 6.06}

Input: 'Please provide me with the most popular video currently trending on YouTube'
Output: {'task_type': 'Summarization', 'scores': 0.21, 'inference_time': 5.58}

